In [17]:
from pysal.model import spreg
import pandas as pd
import geopandas as gpd
import seaborn as sns
import osmnx as ox

from libpysal.weights import lat2W
from libpysal import weights
sns.set(style="whitegrid")

In [18]:
data = pd.read_csv("!NS-SeC.csv")
data.columns

data = data.dropna()

In [19]:
X = data.iloc[:,6:16]
y = data.iloc[:,5]

X['L1, L2 and L3'] = X['L1, L2 and L3']/X['Sum']*100
X['L4, L5 and L6'] = X['L4, L5 and L6']/X['Sum']*100
X['L7'] = X['L7']/X['Sum']*100
X['L8 and L9'] = X['L8 and L9']/X['Sum']*100
X['L10 and L11'] = X['L10 and L11']/X['Sum']*100
X['L12'] = X['L12']/X['Sum']*100
X['L13'] = X['L13']/X['Sum']*100
X['L14.1 and L14.2'] = X['L14.1 and L14.2']/X['Sum']*100
X['L15'] = X['L15']/X['Sum']*100

#X = X.iloc[:,1:]
print(X)
print(y)

      Sum  L1, L2 and L3  L4, L5 and L6         L7  L8 and L9  L10 and L11  \
0    1958       0.014138      30.745659   9.856997  14.044944     4.954035   
2    2095      17.326969      25.441527  12.601432  15.417661     6.921241   
3    1882      13.921360      23.219979  11.742827  10.892667     8.660999   
4    2166       9.510619      19.252078  13.019391  14.496768     5.263158   
5    2020      18.564356      29.405941  11.435644  14.603960     6.534653   
..    ...            ...            ...        ...        ...          ...   
341  1802      12.708102      24.139845  12.319645  12.375139     7.880133   
342  1957      15.738375      22.892182  11.599387  13.030148     7.256004   
343  2155       9.280742      20.232019  11.322506  12.714617     9.095128   
344  2371      20.750738      28.806411  11.809363  11.514129     7.212147   
345  2111      18.285173      26.859308  10.753198  13.074372     6.489815   

           L12        L13  L14.1 and L14.2       L15  
0     5.

In [20]:
variable_names = [
    #"L1, L2 and L3",
    #"L4, L5 and L6",
    "L7",
    #"L8 and L9",
    "L10 and L11",
    "L12",
    #"L13",
    "L14.1 and L14.2",
    "L15"
    ]


# Fit OLS model
m1 = spreg.OLS(
    # Dependent variable
    y.values,
    # Independent variables
    X[variable_names].values,
    # Dependent variable name
    # Independent variable name
    name_x = variable_names,
)

print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     dep_var                Number of Observations:         323
Mean dependent var  :     37.3657                Number of Variables   :           6
S.D. dependent var  :      6.1969                Degrees of Freedom    :         317
R-squared           :      0.5829
Adjusted R-squared  :      0.5763
Sum squared residual:    5157.146                F-statistic           :     88.6120
Sigma-square        :      16.269                Prob(F-statistic)     :   4.329e-58
S.E. of regression  :       4.033                Log likelihood        :    -905.751
Sigma-square ML     :      15.966                Akaike info criterion :    1823.501
S.E of regression ML:      3.9958                Schwarz criterion     :    1846.167

-----------------------------------------------------------------------------

In [21]:
import pyproj
bng=pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

for i in  range(len(data)):
    data.iloc[i,16] = pyproj.transform(bng, wgs84, data.iloc[i,16], data.iloc[i,17])[0]
    data.iloc[i,17] = pyproj.transform(bng, wgs84, data.iloc[i,16], data.iloc[i,17])[1]

from libpysal.weights import KNN
data["geometry"] = gpd.points_from_xy(data['x'], data['y'])

c:\Users\1\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\1\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\1\AppData\Local\Temp\ipykernel_21452\118815056.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  data.iloc[i,16] = pyproj.t

In [22]:
w = KNN.from_dataframe(data, k=16)
m3 = spreg.GM_Lag(y.values, X[variable_names].values, w=w, name_x = variable_names)
print(m3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         323
Mean dependent var  :     37.3657                Number of Variables   :           7
S.D. dependent var  :      6.1969                Degrees of Freedom    :         316
Pseudo R-squared    :      0.5882
Spatial Pseudo R-squared:  0.5811

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      12.6423015       3.7519635       3.3695161       0.0007530
                  L7       0.6214569       0.1575772       3.9438253       0.0000802
         L10 and L11       0.2619737       0.1780349       

In [23]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(y, m1.predy.flatten()),
                  'Lag': mse(y, m3.predy_e)
                    })
mses.sort_values()

OLS    15.966395
Lag    16.035451
dtype: float64

In [24]:
GM = ['Manchester', 'Bolton', 'Bury (England)', 'Oldham (England)', 'Rochdale', 'Salford', 'Stockport', 'Tameside', 'Trafford', 'Wigan']

# get the boundaries of the place (add additional buffer around the query)
boundaries = ox.geocode_to_gdf(GM, buffer_dist=0)

# Let's check the boundaries on a map
boundaries.explore()

C:\Users\1\AppData\Local\Temp\ipykernel_21452\1027148540.py:4: UserWarning: The buffer_dist argument as been deprecated and will be removed in a future release. Buffer your results directly, if desired.
  boundaries = ox.geocode_to_gdf(GM, buffer_dist=0)


ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

In [ ]:
import pyproj
bng=pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

for i in range(len(data)):
    data.iloc[i,17] = pyproj.transform(wgs84, bng, data.iloc[i,17], data.iloc[i,18])[1]
    data.iloc[i,18] = pyproj.transform(wgs84, bng, data.iloc[i,17], data.iloc[i,18])[2]

g:\Python\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
g:\Python\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\11236\AppData\Local\Temp\ipykernel_36732\3234474125.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  data.iloc[i,17] = pyproj.transform(wgs84, b

IndexError: tuple index out of range

In [ ]:


data["x","y"] = pyproj.transform(wgs84, bng, , -2.741771)


data["geometry"] = gpd.points_from_xy(data["x"], data["y"])
print(data)
weights.Queen = weights.Queen.from_dataframe(data)


#data = gpd.GeoDataFrame(data)
#print(data)
# Filter data geographically
#data = gpd.sjoin(data, boundaries[["geometry"]])
#data = data.reset_index(drop=True)

# Check the first rows


KeyError: 'x'

In [ ]:
weights.Queen = weights.Queen.from_dataframe(data)
# Assign spatial lag based on the pool values



AttributeError: 'DataFrame' object has no attribute 'geometry'